In [1]:
import requests
import urllib.request
import time
from bs4 import BeautifulSoup
import bs4
import pandas as pd
import numpy as np
from tqdm import tqdm
import os

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import validators

from selenium.webdriver.common.desired_capabilities import DesiredCapabilities
from selenium import webdriver
import urllib
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By

from tqdm import tqdm

In [33]:
nfl_data = pd.read_csv("./nfl-elo/nfl_elo.csv")
nfl_data['year'] = nfl_data['date'].apply(lambda x: int(x.split("-")[0]))
#nfl_data['month'] = nfl_data['date'].apply(lambda x: int(x.split("-")[1]))

game_data = nfl_data[(nfl_data['year'] == 2010) | \
                     (nfl_data['year'] == 2011) | \
                     (nfl_data['year'] == 2012) | \
                     (nfl_data['year'] == 2013) | \
                     (nfl_data['year'] == 2014) | \
                     (nfl_data['year'] == 2015) | \
                     (nfl_data['year'] == 2016) | \
                     (nfl_data['year'] == 2017) | \
                     (nfl_data['year'] == 2018) | \
                     (nfl_data['year'] == 2019) | \
                     (nfl_data['year'] == 2020) | \
                     (nfl_data['year'] == 2021) | \
                     (nfl_data['year'] == 2022)
                    ]
game_data = game_data[27:]

In [3]:
teams = [
    'BUF',
    'BAL',
    'HOU',
    'DEN',
    'MIA',
    'CIN',
    'IND',
    'KAN',
    'NWE',
    'CLE',
    'JAX',
    'LVR',
    'NYJ',
    'PIT',
    'TEN',
    'LAC',
    'DAL',
    'CHI',
    'ATL',
    'ARI',
    'NYG',
    'DET',
    'CAR',
    'LAR',
    'PHI',
    'GBP',
    'NOS',
    'SFF',
    'WAS',
    'MIN',
    'TBB',
    'SEA'
]

teams_map = {
    'Buffalo Bills': 'BUF',
    'Baltimore Ravens': 'BAL',
    'Houston Texans': 'HOU',
    'Denver Broncos': 'DEN',
    'Miami Dolphins': 'MIA',
    'Cincinnati Bengals': 'CIN',
    'Indianapolis Colts': 'IND',
    'Kansas City Chiefs': 'KAN',
    'New England Patriots': 'NWE',
    'Cleveland Browns': 'CLE',
    'Jacksonville Jaguars': 'JAX',
    'Las Vegas Raiders': 'LVR',
    'Oakland Raiders': 'LVR',
    'New York Jets': 'NYJ',
    'Pittsburgh Steelers': 'PIT',
    'Tennessee Titans': 'TEN',
    'Los Angeles Chargers': 'LAC',
    'San Diego Chargers': 'LAC',
    'Dallas Cowboys': 'DAL',
    'Chicago Bears': 'CHI',
    'Atlanta Falcons': 'ATL',
    'Arizona Cardinals': 'ARI',
    'New York Giants': 'NYG',
    'Detroit Lions': 'DET',
    'Carolina Panthers': 'CAR',
    'Los Angeles Rams': 'LAR',
    'St Louis Rams': 'LAR',
    'Philadelphia Eagles': 'PHI',
    'Green Bay Packers': 'GBP',
    'New Orleans Saints': 'NOS',
    'San Francisco 49ers': 'SFF',
    'Washington Football Team': 'WAS',
    'Washington Redskins': 'WAS',
    'Minnesota Vikings': 'MIN',
    'Tampa Bay Buccaneers': 'TBB',
    'Seattle Seahawks': 'SEA'
}

month_map = {"Jan": 1, "Feb": 2, "Sep": 9, "Oct": 10, "Nov": 11, "Dec": 12}

In [4]:
base_url = "https://www.sportsoddshistory.com/nfl-game-team/?tm="
previous_years = "&d=2010#2019"
valid = True
for t in teams:
    validators.url(base_url + t)
    if(not validators.url(base_url + t)):
        print("{} IS NOT VALID".format(t))
        valid = False
    if(not validators.url(base_url + t + previous_years)):
        print("{} IS NOT VALID".format(t))
        valid = False
        
if(valid):
    print("ALL URLS ARE VALID")

ALL URLS ARE VALID


## Put scraper together for all teams

In [29]:
def scrape_tables(team, tables):
    dates = []
    times = []
    current_teams = []
    opponents = []
    team_scores = []
    opp_scores = []
    game_results = []
    spreads = []
    spread_results = []
    ou_lines = []
    ou_results = []
    for table in tables:
        th_val = len(table.findAll('th'))
        if(th_val in [9,10]):
            for i in range(len(table.findAll('td'))//th_val):
                table_td = table.findAll('td')

                # Get Date and time
                date_td = table_td[i*th_val + 2].contents[0]
                date_str = date_td[-4:] + "-"
                month_str = str(month_map[table.findAll('td')[i*th_val + 2].contents[0][:3]])
                month_str = month_str if(len(month_str) == 2) else '0'+month_str
                date_str += month_str
                date_str += "-" + date_td.split(" ")[1].split(",")[0]
                dates.append(date_str)
                times.append(table_td[i*th_val + 3].contents[0])


                # Get teams
                current_teams.append(team)
                opponents.append(teams_map[table_td[i*th_val + 5].contents[0].contents[0]])

                # Get score and outcome
                game_results.append(table_td[i*th_val + 6].contents[0][0])
                team_scores.append(int(table_td[i*th_val + 6].contents[0][2:].split("-")[0]))
                opp_scores.append(int(table_td[i*th_val + 6].contents[0][2:].split("-")[1].split(" ")[0]))

                # Get spread and outcome
                spread_results.append(table_td[i*th_val + 7].contents[0][0])
                if(table_td[i*th_val + 7].contents[0][2:] == 'PK'):
                    spreads.append(0.)
                else:
                    spreads.append(float(table_td[i*th_val + 7].contents[0][2:]))

                # Get over/under and outcome
                ou_results.append(table_td[i*th_val + 8].contents[0][0])
                ou_lines.append(table_td[i*th_val + 8].contents[0][2:])

    return dates, times, current_teams, opponents, game_results, team_scores, opp_scores, spread_results, spreads, ou_results, ou_lines

In [30]:
dates = []
times = []
current_teams = []
opponents = []
team_scores = []
opp_scores = []
game_results = []
spreads = []
spread_results = []
ou_lines = []
ou_results = []

for idx, t in tqdm(enumerate(teams)):
    url = base_url + teams[0] #+ previous_years
    req = requests.get(url)
    soup = BeautifulSoup(req.content, 'html.parser')
    tables = soup.findAll(lambda tag: tag.name=='table')
    output = scrape_tables(t, tables)

    # Get results
    dates.extend(output[0])
    times.extend(output[1])
    current_teams.extend(output[2])
    opponents.extend(output[3])
    game_results.extend(output[4])
    team_scores.extend(output[5])
    opp_scores.extend(output[6])
    spread_results.extend(output[7])
    spreads.extend(output[8])
    ou_results.extend(output[9])
    ou_lines.extend(output[10])
    print("DONE WITH 2020-2021 {}".format(t))
    
    time.sleep(1)
    url = base_url + teams[0] + previous_years
    req = requests.get(url)
    soup = BeautifulSoup(req.content, 'html.parser')
    tables = soup.findAll(lambda tag: tag.name=='table')
    output = scrape_tables(t, tables)
    
    # Get results
    dates.extend(output[0])
    times.extend(output[1])
    current_teams.extend(output[2])
    opponents.extend(output[3])
    game_results.extend(output[4])
    team_scores.extend(output[5])
    opp_scores.extend(output[6])
    spread_results.extend(output[7])
    spreads.extend(output[8])
    ou_results.extend(output[9])
    ou_lines.extend(output[10])
    print("DONE WITH 2010-2019 {}".format(t))


    time.sleep(5)

0it [00:00, ?it/s]

DONE WITH 2020-2021 BUF
DONE WITH 2011-2019 BUF


1it [00:09,  9.76s/it]

DONE WITH 2020-2021 BAL
DONE WITH 2011-2019 BAL


2it [00:20, 10.28s/it]

...

30it [05:28, 11.89s/it]

DONE WITH 2020-2021 TBB
DONE WITH 2011-2019 TBB


31it [05:37, 11.22s/it]

DONE WITH 2020-2021 SEA
DONE WITH 2011-2019 SEA


32it [05:47, 10.86s/it]


In [41]:
# Save data
np.save("./scraped_data/dates.npy", dates)
np.save("./scraped_data/times.npy", times)
np.save("./scraped_data/current_teams.npy", current_teams)
np.save("./scraped_data/opponents.npy", opponents)
np.save("./scraped_data/game_results.npy", game_results)
np.save("./scraped_data/team_scores.npy", team_scores)
np.save("./scraped_data/opp_scores.npy", opp_scores)
np.save("./scraped_data/spread_results.npy", spread_results)
np.save("./scraped_data/spreads.npy", spreads)
np.save("./scraped_data/ou_results.npy", ou_results)
np.save("./scraped_data/dates.npy", dates)

## Debugging code

In [261]:
url = base_url + teams[0] #+ previous_years
req = requests.get(url)#, headers)
soup = BeautifulSoup(req.content, 'html.parser')
tables = soup.findAll(lambda tag: tag.name=='table')

In [257]:
dates = []
times = []
current_teams = []
opponents = []
team_scores = []
opp_scores = []
game_results = []
spreads = []
spread_results = []
ou_lines = []
ou_results = []
for table in tables:
    th_val = len(table.findAll('th'))
    if(th_val in [9,10]):
        for i in range(len(table.findAll('td'))//th_val):
            table_td = table.findAll('td')
            
            # Get Date and time
            date_td = table_td[i*th_val + 2].contents[0]
            date_str = date_td[-4:] + "-"
            date_str += str(month_map[table.findAll('td')[i*th_val + 2].contents[0][:3]])
            date_str += "-" + date_td.split(" ")[1].split(",")[0]
            dates.append(date_str)
            times.append(table_td[i*th_val + 3].contents[0])

            
            # Get teams
            current_teams.append("BUF")
            opponents.append(teams_map[table_td[i*th_val + 5].contents[0].contents[0]])
            
            # Get score and outcome
            game_results.append(table_td[i*th_val + 6].contents[0][0])
            team_scores.append(int(table_td[i*th_val + 6].contents[0][2:].split("-")[0]))
            opp_scores.append(int(table_td[i*th_val + 6].contents[0][2:].split("-")[1].split(" ")[0]))
            
            # Get spread and outcome
            spread_results.append(table_td[i*th_val + 7].contents[0][0])
            spread.append(float(table_td[i*th_val + 7].contents[0][2:]))
            
            # Get over/under and outcome
            ou_results.append(table_td[i*th_val + 8].contents[0][0])
            ou_lines.append(table_td[i*th_val + 8].contents[0][2:])
            
print(dates)
print(times)
print(opponents)
print(game_results)
print(team_scores)
print(opp_scores)
print(spread_results)
print(spread)
print(ou_results)
print(ou_lines)

['2021-9-12', '2021-9-19', '2021-9-26', '2021-10-3', '2021-10-10', '2021-10-18', '2021-10-31', '2021-11-7', '2021-11-14', '2021-11-21', '2021-11-25', '2021-12-6', '2021-12-12', '2021-12-19', '2021-12-26', '2022-1-2', '2022-1-9', '2022-1-15', '2022-1-23', '2020-9-13', '2020-9-20', '2020-9-27', '2020-10-4', '2020-10-13', '2020-10-19', '2020-10-25', '2020-11-1', '2020-11-8', '2020-11-15', '2020-11-29', '2020-12-7', '2020-12-13', '2020-12-19', '2020-12-28', '2021-1-3', '2021-1-9', '2021-1-16', '2021-1-24']
['1:00', '1:00', '1:00', '1:00', '8:20', '8:15', '1:00', '1:00', '1:00', '1:00', '8:20', '8:15', '4:25', '1:00', '1:00', '1:00', '1:00', '8:15', '6:30', '1:00', '1:00', '1:00', '4:25', '7:00', '5:00', '1:00', '1:00', '1:00', '4:04', '1:00', '8:15', '8:20', '4:30', '8:15', '1:00', '1:05', '8:15', '6:40']
['PIT', 'MIA', 'WAS', 'HOU', 'KAN', 'TEN', 'MIA', 'JAX', 'NYJ', 'IND', 'NOS', 'NWE', 'TBB', 'CAR', 'NWE', 'ATL', 'NYJ', 'NWE', 'KAN', 'NYJ', 'MIA', 'LAR', 'LVR', 'TEN', 'KAN', 'NYJ', 'NWE